In [5]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import re
import csv
from parsel import Selector
import parameter
import datetime
import gender_guesser.detector as gender
from random import *

In [22]:
name = 'Danniele L.'
d = gender.Detector()
first_name = re.match('^\w*', name).group()
sex = d.get_gender(first_name).capitalize()
if re.search('male|Male', sex):
    sex = 'Male'
elif re.search('female|Female', sex):
    sex = 'Female'
else:
    num = randint(0,1)
    if num == 0:
        sex = 'Male'
    else:
        sex = 'Female'
print(sex)

Female


In [3]:
# Parse linkedin profile url
browser = webdriver.Chrome('/bin/chromedriver')
browser.get('https://www.google.com/')
search = browser.find_element_by_name('q')
search.send_keys(parameter.query)
browser.implicitly_wait(10)

search.send_keys(Keys.RETURN)
browser.implicitly_wait(10)

linkedin_urls = []

# for i in range(3):
linkedin_elem = browser.find_elements_by_css_selector('div.r > a:nth-child(1)')
for elem in linkedin_elem:
#   url = re.sub(r"\s›\s", "/in/", elem.text)
    url = elem.get_attribute('href')
    linkedin_urls.append(url)

next_page = browser.find_element_by_id('pnnext') 
next_page.click()   
    
print(linkedin_urls)
# # Login to linkedin
# browser.get('https://www.linkedin.com')
# browser.implicitly_wait(10)

# account = browser.find_element_by_xpath('/html/body/nav/section[2]/form/div[1]/div[1]/input')
# account.send_keys(parameter.linkedin_username)
# browser.implicitly_wait(5)

# password = browser.find_element_by_xpath('/html/body/nav/section[2]/form/div[1]/div[2]/input')
# password.send_keys(parameter.linkedin_password)
# browser.implicitly_wait(5)

# button = browser.find_element_by_xpath('/html/body/nav/section[2]/form/div[2]/button')
# button.click()
# browser.implicitly_wait(10)

# # Visit linkedin profile
# for linkedin_url in linkedin_urls:
#     browser.get(linkedin_url)
#     browser.implicitly_wait(10)
#     print(linkedin_url)


['https://uk.linkedin.com/in/pauljgarner', 'https://uk.linkedin.com/in/bartoszdziendzik', 'https://uk.linkedin.com/in/namakarskiy', 'https://uk.linkedin.com/in/julian-harley-b82b171', 'https://uk.linkedin.com/in/danny-butler-62834777', 'https://uk.linkedin.com/in/oliverernster', 'https://uk.linkedin.com/in/lukaszlechowicz', 'https://uk.linkedin.com/in/nestor-arocha-5869a010', 'https://uk.linkedin.com/in/jeremy-h-00095529', 'https://uk.linkedin.com/in/viktor-m-1295849a']


In [9]:
# Parse all data of a person
browser = webdriver.Chrome('/bin/chromedriver')
# Login to linkedin
browser.get('https://www.linkedin.com')
browser.implicitly_wait(10)

account = browser.find_element_by_xpath('/html/body/nav/section[2]/form/div[1]/div[1]/input')
account.send_keys(parameter.linkedin_username)
browser.implicitly_wait(5)

password = browser.find_element_by_xpath('/html/body/nav/section[2]/form/div[1]/div[2]/input')
password.send_keys(parameter.linkedin_password)
browser.implicitly_wait(5)

button = browser.find_element_by_xpath('/html/body/nav/section[2]/form/div[2]/button')
button.click()
browser.implicitly_wait(5)

# open and extract profile
browser.get('https://www.linkedin.com/in/orhanveli/')
sel = Selector(text=browser.page_source)

# Name
name = browser.find_element_by_css_selector('div.ph5.pb5 > div.display-flex.mt2 > div.flex-1.mr5 > ul.pv-top-card-v3--list.inline-flex.align-items-center  > li.inline.t-24.t-black.t-normal.break-words').text
print(name)

# Gender
d = gender.Detector()
first_name = re.match('^\w*', name).group()
sex = d.get_gender(first_name)
print(sex)

# JobRole
current_job = browser.find_element_by_css_selector('div.ph5.pb5 > div.display-flex.mt2 > div.flex-1.mr5 > h2').text
print(current_job)

# Education
uni = browser.find_element_by_css_selector('div.pv-entity__summary-info.pv-entity__summary-info--background-section > div > h3').text
degree = browser.find_element_by_css_selector('div.pv-entity__summary-info.pv-entity__summary-info--background-section > div > p.pv-entity__secondary-title.pv-entity__degree-name.t-14.t-black.t-normal > span.pv-entity__comma-item').text
department = browser.find_element_by_css_selector('div.pv-entity__summary-info.pv-entity__summary-info--background-section > div > p.pv-entity__secondary-title.pv-entity__fos.t-14.t-black.t-normal > span.pv-entity__comma-item').text
time = browser.find_element_by_css_selector('div.pv-entity__summary-info.pv-entity__summary-info--background-section > p > span:nth-child(2)').text
print(uni + '\n' + degree + '\n' + department + '\n' + time)

# Experience
experience = []
# exp_element = browser.find_elements_by_id('experience-section')
exp_element = browser.find_elements_by_css_selector('#experience-section ul .pv-position-entity')

def if_company_list(element):
    return element.find_elements_by_css_selector('.pv-entity__role-details-container')

for ele in exp_element:
    if(if_company_list(ele)):
        job_title, period = [], []
        company = ele.find_element_by_css_selector('.pv-entity__company-summary-info > h3 > span:nth-of-type(2)').text
        job_title_element = ele.find_elements_by_css_selector('.pv-entity__summary-info-v2 > h3 > span:nth-of-type(2)')
        period_element = ele.find_elements_by_class_name('pv-entity__bullet-item-v2')
        for element in job_title_element:
            job_title.append(element.text)
        for element in period_element:
            period.append(element.text)
            
        position = dict([
            ('company', company),
            ('job_title', job_title),
            ('period', period)
        ])
        experience.append(position)
                                                   
    else:
        company = ele.find_element_by_class_name('pv-entity__secondary-title').text
        job_title = ele.find_element_by_css_selector('.pv-entity__summary-info h3:nth-of-type(1)').text
        period = ele.find_element_by_class_name('pv-entity__bullet-item-v2').text

        position = dict([
            ('company', company),
            ('job_title', job_title),
            ('period', period)
            ])
        experience.append(position)

print(experience)
writer = csv.writer(open(parameter.file_name, 'w', newline=''))
with open('result.csv', mode='w') as result:
    writer = csv.writer(result, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)

    writer.writerow(['name','current_job','uni','degree', 'department','time', 'pic', 'experience'])

    writer.writerow([name,
                 current_job,
                 uni,
                 degree,
                 department,
                 time,
                 pic,
                 ])

# Age *
# Education *
# NumCompaniesWorked *
# TotalWorkingYears *
# JobLevel **input
# Gender *
# EducationField *
# Department **input
# JobRole **input

Orhan Veli Firik
male
Front-end JavaScript, TypeScript, React Developer (Open for Contracts)
Ege University
Bachelor's degree
Faculty of Pharmacy
2000 年 – 2006 年
https://media.licdn.com/dms/image/C4D03AQGvFctc9KOwDQ/profile-displayphoto-shrink_200_200/0?e=1581552000&v=beta&t=FmBX2UD0yU3AIRnUgStvSd1XBJfSKEcrfb0habeV-HY
[{'company': 'Elsewhen', 'job_title': 'Frontend Web Developer', 'period': '2 個月'}, {'company': 'Spike Web Solutions', 'job_title': 'Founder', 'period': '3 個月'}, {'company': 'Ekşi Teknoloji', 'job_title': 'Web Developer', 'period': '3 年 2 個月'}, {'company': 'Netmarble Turkey', 'job_title': ['Web Developer, Serior Front-end Developer', 'Web Developer / Front-end Developer'], 'period': ['5 年 11 個月', '5 年']}, {'company': 'www.dw.gen.tr', 'job_title': 'Web Developer & Project Owner', 'period': '9 年'}]


In [30]:
# create and write csv test 
with open('result.csv', mode='w') as result:
    writer = csv.writer(result, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    writer.writerow(['name', 'current_job', 'uni', 'degree', 'department', 'time', 'pic', 'experience'])
    
with open('result.csv', 'a') as f:
    writer = csv.writer(f)
    writer.writerow(['name', 'current_job', 'uni', 'degree', 'department', 'time', 'pic', 'experience'])

In [2]:
# Test save experience to csv
browser = webdriver.Chrome('/bin/chromedriver')
# Login to linkedin
browser.get('https://www.linkedin.com')
browser.implicitly_wait(10)

account = browser.find_element_by_xpath('/html/body/nav/section[2]/form/div[1]/div[1]/input')
account.send_keys(parameter.linkedin_username)
browser.implicitly_wait(5)

password = browser.find_element_by_xpath('/html/body/nav/section[2]/form/div[1]/div[2]/input')
password.send_keys(parameter.linkedin_password)
browser.implicitly_wait(5)

button = browser.find_element_by_xpath('/html/body/nav/section[2]/form/div[2]/button')
button.click()
browser.implicitly_wait(5)

# open and extract profile
browser.get('https://www.linkedin.com/in/anairen-rodriguez-b40200132/')
sel = Selector(text=browser.page_source)

# Experience
experience = []
exp_element = browser.find_elements_by_css_selector('#experience-section ul .pv-position-entity')

def if_company_list(element):
    return element.find_elements_by_css_selector('.pv-entity__role-details-container')

for ele in exp_element:
    if(if_company_list(ele)):
        job_title, period = [], []
        company = ele.find_element_by_css_selector('.pv-entity__company-summary-info > h3 > span:nth-of-type(2)').text
        job_title_element = ele.find_elements_by_css_selector('.pv-entity__summary-info-v2 > h3 > span:nth-of-type(2)')
        period_element = ele.find_elements_by_class_name('pv-entity__bullet-item-v2')
        for element in job_title_element:
            job_title.append(element.text)
        for element in period_element:
            period.append(element.text)
            
        position = dict([
            ('company', company),
            ('job_title', job_title),
            ('period', period)
        ])
        experience.append(position)
                                                   
    else:
        company = ele.find_element_by_class_name('pv-entity__secondary-title').text
        job_title = ele.find_element_by_css_selector('.pv-entity__summary-info h3:nth-of-type(1)').text
        period = ele.find_element_by_class_name('pv-entity__bullet-item-v2').text

        position = dict([
            ('company', company),
            ('job_title', job_title),
            ('period', period)
            ])
        experience.append(position)

NumCompaniesWorked = len(experience)
print(experience)
writer = csv.writer(open(parameter.file_name, 'w', newline=''))
with open('exp.csv', mode='w') as result:
    writer = csv.writer(result, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    writer.writerow(['experience'])
    writer.writerow(experience)

[{'company': 'European Union Center at the University of Miami', 'job_title': 'Center Coordinator', 'period': '5 個月'}, {'company': 'America Reads*America Counts', 'job_title': 'Tutor', 'period': '3 年'}, {'company': 'University of Miami', 'job_title': 'Research Assistant at the Department of International Studies', 'period': '3 個月'}]


In [3]:
# Count working year
working_year = 0
working_month = 0
for exp in experience:
    period = exp.get('period')
    if isinstance(period, list):
        for elem in period:
            if(re.search('\d*\s[年]', elem) != None):
                year = re.search('\d*\s[年]', elem).group()
                num_of_year = int(re.search('^\d*', year).group())
                working_year += (num_of_year)
            if(re.search('\d*\s[個月]', elem) != None):
                month = re.search('\d*\s[個月]', elem).group()
                num_of_month = int(re.search('^\d*', month).group())
                working_month += (num_of_month)
    else:
        if(re.search('\d*\s[年]', period) != None):
            year = re.search('\d*\s[年]', period).group()
            num_of_year = int(re.search('^\d*', year).group())
            working_year += (num_of_year)
        if(re.search('\d*\s[個月]', period) != None):
            month = re.search('\d*\s[個月]', period).group()
            num_of_month = int(re.search('^\d*', month).group())
            working_month += (num_of_month)

print('Total year is ' + str(working_year) + ' and total month is ' + str(working_month))  

month_to_year = working_month//12
TotalWorkingYears = working_year + month_to_year
print(TotalWorkingYears)

Total year is 3 and total month is 8
3


In [5]:
# Test get education and age
def educaton_filter(degree):
    if (re.search('Bachelor|B.*S.*|B.*A.*', degree) != None):
        return (3)
    elif (re.search('Master|M.*B.*A.*|', degree) != None):
        return (4)
    else:
        return(degree)
    
def get_age(time):
    year = re.match('^\d{4}', time)
    if (year != None):
        age = 2019 - int(year.group()) + 18
        return(age)
    else:
        return time
    
def educationfield_filter(department):
    marketing = re.search('business|management|finance|economics', department, re.IGNORECASE)
    technical = re.search('techonology|engineering|computer', department, re.IGNORECASE)
    life = re.search('psychology|engineering|computer', department, re.IGNORECASE)
    human_resource = re.search('human resource', department, re.IGNORECASE)
    medical = re.search('medical|pharmacy', department, re.IGNORECASE)
    
    if (marketing != None):
        return('Marketing')
    elif (technical != None):
        return('Technical Degree')
    elif (life != None):
        return('Life Sciences')
    elif (human_resource != None):
        return('Human Resources')
    elif (medical != None):
        return('Medical')
    else:
        return('Other')
    
browser = webdriver.Chrome('/bin/chromedriver')
# Login to linkedin
browser.get('https://www.linkedin.com')
browser.implicitly_wait(10)

account = browser.find_element_by_xpath('/html/body/nav/section[2]/form/div[1]/div[1]/input')
account.send_keys(parameter.linkedin_username)
browser.implicitly_wait(5)

password = browser.find_element_by_xpath('/html/body/nav/section[2]/form/div[1]/div[2]/input')
password.send_keys(parameter.linkedin_password)
browser.implicitly_wait(5)

button = browser.find_element_by_xpath('/html/body/nav/section[2]/form/div[2]/button')
button.click()
browser.implicitly_wait(5)

# open and extract profile
browser.get('https://www.linkedin.com/in/anairen-rodriguez-b40200132/')
sel = Selector(text=browser.page_source)

# Name
name = browser.find_element_by_css_selector('div.ph5.pb5 > div.display-flex.mt2 > div.flex-1.mr5 > ul.pv-top-card-v3--list.inline-flex.align-items-center  > li.inline.t-24.t-black.t-normal.break-words').text
print(name)

# JobRole
current_job = browser.find_element_by_css_selector('div.ph5.pb5 > div.display-flex.mt2 > div.flex-1.mr5 > h2').text
# print(current_job)

# Education
uni = browser.find_element_by_css_selector('div.pv-entity__summary-info.pv-entity__summary-info--background-section > div > h3').text
degree = browser.find_element_by_css_selector('div.pv-entity__summary-info.pv-entity__summary-info--background-section > div > p.pv-entity__secondary-title.pv-entity__degree-name.t-14.t-black.t-normal > span.pv-entity__comma-item').text
department = browser.find_element_by_css_selector('div.pv-entity__summary-info.pv-entity__summary-info--background-section > div > p.pv-entity__secondary-title.pv-entity__fos.t-14.t-black.t-normal > span.pv-entity__comma-item').text
time = browser.find_element_by_css_selector('div.pv-entity__summary-info.pv-entity__summary-info--background-section > p > span:nth-child(2)').text
print(time)

education = educaton_filter(degree)
print(education)
age = get_age(time)
print(age)
EducationField = educationfield_filter(department)
print(EducationField)

Anairen Rodriguez
2016 年 – 2020 年
3
21
Other
